Тут мы реализуем сервис записи фичей в новую таблицу для того чтобы не загружать основной алгоритм обработкой данных

# Загрузка фичей из базы данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine


def load_and_merge_data(engine, chunksize=200000):
    # Чтение данных таблицы user_data
    query = "SELECT * FROM user_data"
    user_data = pd.read_sql(query, engine)
    print(f"User data shape: {user_data.shape}")

    # Чтение данных таблицы post_text_df
    query = "SELECT * FROM post_text_df"
    post_text_df = pd.read_sql(query, engine)
    print(f"Post text data shape: {post_text_df.shape}")

    # Чтение ограниченного количества данных таблицы feed_data
    query = f"SELECT * FROM feed_data"
    feed_data = batch_load_sql_timed(engine, query, chunksize)
    print(f"Feed data shape: {feed_data.shape}")

    # Переименование столбцов идентификаторов
    user_data = user_data.rename(columns={'id': 'user_id'})
    post_text_df = post_text_df.rename(columns={'id': 'post_id'})

    # Объединение таблиц
    data = feed_data.merge(user_data, on='user_id', how='left')
    data = data.merge(post_text_df, on='post_id', how='left')

    print(f"Data shape after load_and_merge_data: {data.shape}")

    return data


def batch_load_sql(engine, query: str, chunksize: int) -> pd.DataFrame:
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=chunksize):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

import time

def batch_load_sql_timed(engine, query: str, chunksize: int) -> pd.DataFrame:
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    row_count = 0
    start_time = time.time()

    for chunk_dataframe in pd.read_sql(query, conn, chunksize=chunksize):
        chunks.append(chunk_dataframe)
        row_count += len(chunk_dataframe)
        print(f"Loaded {row_count} rows, elapsed time: {time.time() - start_time:.2f} seconds")

    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import time

In [2]:
engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )

chunksize = 1000000

In [3]:
# Чтение данных таблицы user_data
query = "SELECT * FROM user_data"
user_data = pd.read_sql(query, engine)
print(f"User data shape: {user_data.shape}")

User data shape: (163205, 8)


In [4]:
# Чтение данных таблицы post_text_df
query = "SELECT * FROM post_text_df"
post_text_df = pd.read_sql(query, engine)
print(f"Post text data shape: {post_text_df.shape}")

Post text data shape: (7023, 3)


In [5]:
import time

def batch_load_sql_timed(engine, query: str, chunksize: int) -> pd.DataFrame:
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    row_count = 0
    start_time = time.time()

    for chunk_dataframe in pd.read_sql(query, conn, chunksize=chunksize):
        chunks.append(chunk_dataframe)
        row_count += len(chunk_dataframe)
        print(f"Loaded {row_count} rows, elapsed time: {time.time() - start_time:.2f} seconds")

    conn.close()
    return pd.concat(chunks, ignore_index=True)

In [6]:
# Чтение ограниченного количества данных таблицы feed_data
query = f"SELECT * FROM feed_data LIMIT 1000000"
feed_data = batch_load_sql_timed(engine, query, chunksize)
print(f"Feed data shape: {feed_data.shape}")

Loaded 1000000 rows, elapsed time: 111.40 seconds
Feed data shape: (1000000, 5)


In [10]:
# Переименование столбцов идентификаторов
user_data = user_data.rename(columns={'id': 'user_id'})
post_text_df = post_text_df.rename(columns={'id': 'post_id'})

# Объединение таблиц
data = feed_data.merge(user_data, on='user_id', how='left')
data = data.merge(post_text_df, on='post_id', how='left')

print(f"Data shape after load_and_merge_data: {data.shape}")

Data shape after load_and_merge_data: (1000000, 14)


# Обработка временных меток

In [11]:
# Преобразование формата временных меток в объект datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Извлечение признаков из временных меток
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['hour_of_day'] = data['timestamp'].dt.hour

# Расчет времени с момента последнего действия для каждого пользователя
data = data.sort_values(['user_id', 'timestamp'])
data['time_since_last_action'] = data.groupby('user_id')['timestamp'].diff().dt.total_seconds()
data['time_since_last_action'].fillna(0, inplace=True)

# Удаление столбца временных меток
data = data.drop('timestamp', axis=1)

print('Timestamps processed')
print(f"Data shape after timestamps processing: {data.shape}")

Timestamps processed
Data shape after timestamps processing: (1000000, 16)


# Создание дополнительных признаков

In [12]:
# Feature 1: Количество просмотров и лайков для каждого пользователя
user_views_likes = data.groupby('user_id')['action'].value_counts().unstack().fillna(0)
user_views_likes.columns = ['user_views', 'user_likes']
data = data.merge(user_views_likes, on='user_id', how='left')

# Feature 2: Количество просмотров и лайков для каждого поста
post_views_likes = data.groupby('post_id')['action'].value_counts().unstack().fillna(0)
post_views_likes.columns = ['post_views', 'post_likes']
data = data.merge(post_views_likes, on='post_id', how='left')

# Feature 3: Количество просмотров и лайков для каждой группы тематик
temp_df = data[['exp_group', 'topic', 'action']]

# Создание колонок с количеством просмотров и лайков для каждой темы внутри группы
topic_action_count = temp_df.pivot_table(index='exp_group', columns=['topic', 'action'], aggfunc=len, fill_value=0)
topic_action_count.columns = [f'{col[0]}_exp_group_{col[1]}s' for col in topic_action_count.columns]
grouped_data = topic_action_count.reset_index()

data = data.merge(grouped_data, on='exp_group', how='left')

# Преобразование категориальных признаков в строковый формат
categorical_columns = ['country', 'city', 'topic', 'gender', 'os', 'source']
data[categorical_columns] = data[categorical_columns].astype(str)

print('Additional features created')
print(f"Data shape after additional features creation: {data.shape}")

Additional features created
Data shape after additional features creation: (1000000, 34)


In [17]:
data.to_sql('my_favourite_table_69', con=engine) # записываем таблицу

ValueError: Table 'my_favourite_table_69' already exists.

In [18]:
df = pd.read_sql('SELECT * FROM my_favourite_table_69 LIMIT 1000', con=engine) # считываем таблицу

In [19]:
df.head()

,index,user_id,post_id,action,target,gender,age,country,city,exp_group,...,entertainment_exp_group_likes,entertainment_exp_group_views,movie_exp_group_likes,movie_exp_group_views,politics_exp_group_likes,politics_exp_group_views,sport_exp_group_likes,sport_exp_group_views,tech_exp_group_likes,tech_exp_group_views


In [20]:
df.shape()

TypeError: 'tuple' object is not callable

# Подготовка данных для инференса

In [ ]:
# Убираем ненужные столбцы
X = data.drop(['target', 'action', 'text'], axis=1)

categorical_columns = ['country', 'topic', 'city', 'gender', 'os', 'source']

# Создание ID группы на основе столбца 'user_id'
unique_user_ids = X['user_id'].unique()
group_id_dict = {user_id: idx for idx, user_id in enumerate(unique_user_ids)}
X['group_id'] = X['user_id'].map(group_id_dict)

# Сортировка набора данных для предсказаний по 'group_id'
X = X.sort_values(by='group_id')

# Убедитесь, что категориальные переменные представлены в виде строк
X[categorical_columns] = X[categorical_columns].astype(str)



# Запись фичей в базу данных

# То что будет в сервисе

In [ ]:
from catboost import Pool

## TODO: надо просто передать лист с индексами категориальных признаков
# Получение индексов категориальных столбцов
cat_features = [X.drop(columns=['user_id']).columns.get_loc(col) for col in categorical_columns]

# Создание объекта Pool для набора данных предсказаний с колонкой 'group_id' и категориальными признаками
prediction_pool = Pool(X.drop(columns=['user_id']), cat_features=cat_features, group_id=X['group_id'])